## Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Preparing Foursquare API

In [2]:
CLIENT_ID = 'JMUJGUOWLCEIRDXPN0TXF2SOPD4IG1FZWSKDIU3KHPUR3NFM' # your Foursquare ID
CLIENT_SECRET = 'U4F5YPU0QAVSIW1RX5JBENSJDP3M304VT51FL04B5ITMTZWN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
#LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: JMUJGUOWLCEIRDXPN0TXF2SOPD4IG1FZWSKDIU3KHPUR3NFM


## Select top 6 Russian cities and Colleges and Cafes there

In [3]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["Moscow, RU", 'Saint Petersburg, RU', 'Novosibirsk, RU', 'Yekaterinburg, RU', 'Kazan, RU', 'Nizhny Novgorod, RU']
results_cafe = {}
results_college = {}
for city in cities:
    url_cafe = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d16d941735") # Cafe CATEGORY ID
    results_cafe[city] = requests.get(url_cafe).json()
    url_college = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4d4b7105d754a06372d81259") # College CATEGORY ID
    results_college[city] = requests.get(url_college).json()

## Select Name, Address, Lat, Lng for College/Cafe from JSON file and store it in dataframes

In [4]:
df_venues_cafe={}
df_venues_college={}
for city in cities:
    venues_cafe = pd.json_normalize(results_cafe[city]['response']['groups'][0]['items'])
    df_venues_cafe[city] = venues_cafe[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues_cafe[city].columns = ['Name', 'Address', 'Lat', 'Lng']
    venues_college = pd.json_normalize(results_college[city]['response']['groups'][0]['items'])
    df_venues_college[city] = venues_college[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues_college[city].columns = ['Name', 'Address', 'Lat', 'Lng']

## Calculate Cafe / College ratio for all cities and get maps for college

In [16]:
from math import sqrt
maps = {}
for city in cities:
    city_lat = np.mean([results_college[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results_college[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results_college[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results_college[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for latcl, lngcl, labelcl  in zip(df_venues_cafe[city]['Lat'], df_venues_cafe[city]['Lng'], df_venues_cafe[city]['Name']):
        labelcl = folium.Popup(labelcl, parse_html=True)
        folium.CircleMarker(
            [latcl, lngcl],
            radius=5,
            popup=labelcl,
            color='red',
            fill=True,
            fill_color='#cc8631',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    #print(f"Total number of cafe places in {city} = ", results_cafe[city]['response']['totalResults'])
    print(f"Total number of college places in {city} = ", results_college[city]['response']['totalResults'])
    print(f"Cafes per college in {city} = ", results_cafe[city]['response']['totalResults']/results_college[city]['response']['totalResults'])

Total number of college places in Moscow, RU =  224
Cafes per college in Moscow, RU =  1.375
Total number of college places in Saint Petersburg, RU =  220
Cafes per college in Saint Petersburg, RU =  1.3272727272727274
Total number of college places in Novosibirsk, RU =  46
Cafes per college in Novosibirsk, RU =  2.4347826086956523
Total number of college places in Yekaterinburg, RU =  76
Cafes per college in Yekaterinburg, RU =  1.618421052631579
Total number of college places in Kazan, RU =  100
Cafes per college in Kazan, RU =  1.35
Total number of college places in Nizhny Novgorod, RU =  63
Cafes per college in Nizhny Novgorod, RU =  1.9047619047619047


In [23]:
maps[cities[5]]

## Calculate College/cafe distances and filter Colleges with no Cafe in 10-min walk. Build a map

In [7]:
maps2 = {}
df_venues_college2 = {}

for city in cities:
    filter_arr = []
    city_lat = np.mean([results_college[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results_college[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results_college[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results_college[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps2[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    
    # add markers to map
    numcollegenocafe = 0
    index = 0
    for latcl, lngcl, labelcl  in zip(df_venues_college[city]['Lat'], df_venues_college[city]['Lng'], df_venues_college[city]['Name']):
        numcafe = 0
        for latcf, lngcf, labelcf  in zip(df_venues_cafe[city]['Lat'], df_venues_cafe[city]['Lng'], df_venues_cafe[city]['Name']):
            dist=sqrt((latcl-latcf)*(latcl-latcf)+(lngcl-lngcf)*(lngcl-lngcf))
            if (dist < 0.01):
                numcafe = numcafe + 1
        if (numcafe==0):
            filter_arr.append(True) 
            numcollegenocafe = numcollegenocafe + 1
            labelcl = folium.Popup(labelcl, parse_html=True)
            folium.CircleMarker(
                [latcl, lngcl],
                radius=5,
                popup=labelcl,
                color='blue',
                fill=True,
                fill_color='#3186cc',
                fill_opacity=0.7,
                parse_html=False).add_to(maps2[city])
        else:
            filter_arr.append(False)
        index = index + 1
    print(f"Total number of college places without cafe in {city} = ", numcollegenocafe)
    print(f"Percent of college places without cafe in {city} = ", numcollegenocafe*100.0/results_college[city]['response']['totalResults'])
    df_venues_college2[city] = df_venues_college[city][filter_arr]

Total number of college places without cafe in Moscow, RU =  56
Percent of college places without cafe in Moscow, RU =  25.0
Total number of college places without cafe in Saint Petersburg, RU =  38
Percent of college places without cafe in Saint Petersburg, RU =  17.272727272727273
Total number of college places without cafe in Novosibirsk, RU =  11
Percent of college places without cafe in Novosibirsk, RU =  23.91304347826087
Total number of college places without cafe in Yekaterinburg, RU =  9
Percent of college places without cafe in Yekaterinburg, RU =  11.842105263157896
Total number of college places without cafe in Kazan, RU =  17
Percent of college places without cafe in Kazan, RU =  17.0
Total number of college places without cafe in Nizhny Novgorod, RU =  6
Percent of college places without cafe in Nizhny Novgorod, RU =  9.523809523809524


In [8]:
maps2[cities[0]]

## Prepare data for clustering colleges in Moscow

In [9]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 24

df_venues_college_clustering = df_venues_college2['Moscow, RU'].drop('Address',1).drop('Name',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_venues_college_clustering)

# check cluster labels generated for each row in the dataframe
df_venues_college2['Moscow, RU'].insert(0, 'Cluster Labels', kmeans.labels_)

kmeans.labels_[0:57] 

array([ 7,  3, 14,  9, 15, 23, 22,  8, 19,  0, 13, 15,  0, 18,  2, 16,  0,
        8,  5, 12,  0,  5,  9,  2,  0, 20,  6, 20,  9, 11,  0,  0, 10,  0,
       19,  2,  6, 21,  6,  2,  1, 17,  8,  0,  0,  2,  0,  8,  4, 22, 19,
        6,  6,  6, 13, 17], dtype=int32)

## Cluster colleges in Moscow and build a map of clusters

In [10]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors


city_lat = np.mean([results_college['Moscow, RU']['response']['geocode']['geometry']['bounds']['ne']['lat'],
                    results_college['Moscow, RU']['response']['geocode']['geometry']['bounds']['sw']['lat']])
city_lng = np.mean([results_college['Moscow, RU']['response']['geocode']['geometry']['bounds']['ne']['lng'],
                    results_college['Moscow, RU']['response']['geocode']['geometry']['bounds']['sw']['lng']])
map_clusters = folium.Map(location=[city_lat, city_lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_venues_college2['Moscow, RU']['Lat'], df_venues_college2['Moscow, RU']['Lng'], df_venues_college2['Moscow, RU']['Name'], df_venues_college2['Moscow, RU']['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (cluster == 0):
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color='black',
            fill=True,
            fill_color='black',
            fill_opacity=0.7).add_to(map_clusters)
    else:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters